In [35]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import pandas as pd
import seaborn as sns
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

In [36]:
np.random.seed(42)

In [37]:
def count_support_vectors(a, C):
    l = len(a) // 2
    count = 0

    for i in range(l):
        if 0 < a[i] < C or 0 < a[i + l] < C:
            count += 1

    return count

def extract_support_vectors(a, x_train, C):
    l = len(x_train)
    support_vector_indices = []

    for i in range(l):
        if 0 < a[i] < C or 0 < a[i + l] < C:
            support_vector_indices.append(i)

    support_vectors = x_train[support_vector_indices]
    return support_vector_indices, support_vectors
class Info:
    def __init__(self, size):
        self.history_of_norm = np.zeros(size)

    def show_history_of_norm(self):
        plt.plot(np.log10(self.history_of_norm))
        plt.grid()
        plt.title("History of norm")
        plt.show()


def sinc_function(x):
    return np.sinc(x)
    # return np.sin(np.pi * x / 4.0) + 0.5 * np.sin(np.pi * x)

PREPARE DATA

In [38]:
l = 200
x_min, x_max = -5, 5
x = np.linspace(x_min, x_max, l)
y_true = sinc_function(x)

noise = np.random.normal(0, 0.1, l)
y_noisy = y_true + noise

In [39]:
def rbf_kernel(x1, x2, gamma):
    return np.exp(-gamma * np.linalg.norm(x1 - x2) ** 2)


def compute_kernel_matrix(X, gamma):
    n = len(X)
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i, j] = rbf_kernel(X[i], X[j], gamma)
    return K


def sor_algorithm(H, E, c, C, tolerance=1e-3, max_iter=10000, omega=1.0):
    info = Info(max_iter)
    A = H + E
    a = np.zeros_like(c)

    # Разложение A = L + D + L^T
    D = np.diag(np.diag(A))  # Диагональная часть
    L = np.tril(A, k=-1)  # Строго нижняя треугольная часть

    diag = np.diag(D)
    if np.any(diag == 0):
        raise ValueError("Diagonal elements of D contain zeros!")
    diag_inv = 1 / diag

    for iteration in range(max_iter):
        a_prev = a.copy()

        for j in range(len(a)):
            sum_L = np.dot(L[j, :j], a[:j])  # Вклад от строго нижней треугольной части
            sum_D_U = np.dot(A[j, j:], a_prev[j:])  # Вклад от диагональной и верхней частей
            a[j] = a_prev[j] - omega * diag_inv[j] * (sum_L + sum_D_U - c[j])
            a[j] = max(0, min(C, a[j]))  # Проекция на [0, C]

        norm = np.linalg.norm(a - a_prev)
        info.history_of_norm[iteration] = norm
        if norm < tolerance:
            break

    return a, info

In [40]:
def predict(x_new, x_train, a, b, gamma):
    n = len(x_train)
    y_pred = np.zeros(len(x_new))
    for i in range(len(x_new)):
        y_pred[i] = b
        for j in range(n):
            y_pred[i] += (a[j] - a[j + n]) * rbf_kernel(x_train[j], x_new[i], gamma)
    return y_pred

In [42]:
from joblib import Parallel, delayed

param_grid = {
    'C': [0.1, 1, 2, 5, 7, 10],          # 5 значений для C (например, [0.1, 0.46, 2.15, 10, 46.4])
    'epsilon': np.linspace(0.07, 0.25, 6),  # 5 значений для epsilon (например, [0.01, 0.06, 0.11, 0.16, 0.2])
    'gamma': [0.1, 0.7, 1, 1.5, 3, 10]        # 5 значений для gamma (например, [0.1, 0.32, 1, 3.2, 10])
}

# Функция для выполнения вычислений для одного значения omega
def process_omega(omega):
    results = []  # Список для хранения результатов текущей omega
    best_mae = float('inf')
    best_params = None
    best_info = None

    total_iterations = len(list(ParameterGrid(param_grid)))
    with tqdm(total=total_iterations, desc=f"Grid Search Progress (omega={omega})", unit="iteration") as pbar:
        for params in ParameterGrid(param_grid):
            C, epsilon, gamma = params['C'], params['epsilon'], params['gamma']

            K = compute_kernel_matrix(x, gamma)
            H = np.block([[K, -K], [-K, K]])
            E = np.block([[np.ones((l, l)), -np.ones((l, l))], [-np.ones((l, l)), np.ones((l, l))]])
            c = np.concatenate([y_noisy - epsilon, -y_noisy - epsilon])

            # Передаем omega в sor_algorithm
            a, info = sor_algorithm(H, E, c, C, omega=omega)
            b = -np.sum(a[l:] - a[:l])

            y_pred = predict(x, x, a, b, gamma)

            mae = mean_absolute_error(y_true, y_pred)
            sv_count = count_support_vectors(a, C)

            results.append({
                'C': C,
                'epsilon': epsilon,
                'gamma': gamma,
                'sv_count': sv_count,
                'MAE': mae,
                'omega': omega  # Добавляем omega в результаты
            })

            if mae < best_mae:
                best_mae = mae
                best_params = params
                best_info = info

            pbar.update(1)

    # Преобразуем результаты в DataFrame
    results_df = pd.DataFrame(results)

    # Сохраняем результаты в CSV-файл
    filename = f'results_omega_{omega}.csv'
    results_df.to_csv(filename, index=False)
    print(f"Results for omega={omega} saved to {filename}")

    # Выводим таблицу с результатами
    print(f"Results for omega={omega}:")
    print(results_df)

    # Выводим лучшие параметры
    print(f"Best MAE for omega={omega}: {best_mae:.4f}")
    print(f"Best parameters for omega={omega}: {best_params}")

    return results_df

# Запускаем параллельные вычисления
omega_values = [0.5, 1.0, 1.5, 1.9]
results_dfs = Parallel(n_jobs=-1)(delayed(process_omega)(omega) for omega in omega_values)



In [9]:
def extract_bounded_support_vectors(a, x_train, C):
    """
    Выделяет связные векторы (граничные опорные векторы).
    :param a: Массив двойственных переменных (размер 2*l).
    :param x_train: Обучающие данные (размер l).
    :param C: Параметр регуляризации.
    :return: Индексы связных векторов и их координаты.
    """
    l = len(x_train)
    bounded_support_vector_indices = []

    for i in range(l):
        if np.isclose(a[i], C) or np.isclose(a[i + l], C):  # Проверяем условия для a_i и a_i^*
            bounded_support_vector_indices.append(i)

    bounded_support_vectors = x_train[bounded_support_vector_indices]
    return bounded_support_vector_indices, bounded_support_vectors

In [20]:
# Выбираем omega=1.0 и лучшие гиперпараметры
omega = 1.0
best_params = {'C': 3.1622776601683795, 'epsilon': 0.01, 'gamma': 1.0}
C_best, epsilon_best, gamma_best = best_params['C'], best_params['epsilon'], best_params['gamma']

# Вычисляем матрицы и решаем задачу для лучших параметров
K_best = compute_kernel_matrix(x, gamma_best)
H_best = np.block([[K_best, -K_best], [-K_best, K_best]])
E_best = np.block([[np.ones((l, l)), -np.ones((l, l))], [-np.ones((l, l)), np.ones((l, l))]])
c_best = np.concatenate([y_noisy - epsilon_best, -y_noisy - epsilon_best])
a_best, info_best = sor_algorithm(H_best, E_best, c_best, C_best, omega=omega)
b_best = -np.sum(a_best[l:] - a_best[:l])

# Используем x_test для плавного графика
x_test = np.linspace(x_min, x_max, 500)
y_pred_best = predict(x_test, x, a_best, b_best, gamma_best)

# Извлекаем опорные и связные векторы
sup_vectors_indices, _ = extract_support_vectors(a_best, x, C_best)
bounded_sup_vectors_indices, _ = extract_bounded_support_vectors(a_best, x, C_best)

# Количество опорных и связных векторов
num_support_vectors = len(sup_vectors_indices)
num_bounded_support_vectors = len(bounded_sup_vectors_indices)

# Строим график
plt.figure(figsize=(10, 6))
plt.plot(x, y_true, label="True function", color="black", linestyle="--")
plt.scatter(x, y_noisy, label="Noisy data", color="blue", alpha=0.5)

# Отображаем опорные векторы
plt.scatter(x[sup_vectors_indices], y_noisy[sup_vectors_indices], color="red", alpha=0.9,
            label=f"Support vectors ({num_support_vectors})")

# Отображаем связные векторы
plt.scatter(x[bounded_sup_vectors_indices], y_noisy[bounded_sup_vectors_indices], color="green", alpha=0.9,
            label=f"Bounded support vectors ({num_bounded_support_vectors})")

plt.plot(x_test, y_pred_best, label="Predicted function", color="red")
plt.fill_between(x_test, y_pred_best - epsilon_best, y_pred_best + epsilon_best, color="gray", alpha=0.2,
                 label="ε-tube")

# Указываем количество опорных и связных векторов в легенде
plt.legend()
plt.title("Support Vector Regression with Best Parameters and Bounded Vectors")
plt.xlabel("x")
plt.ylabel("y")

# Выводим количество опорных и связных векторов под графиком
plt.figtext(0.5, -0.1,
            f"Number of Support Vectors: {num_support_vectors}\n"
            f"Number of Bounded Support Vectors: {num_bounded_support_vectors}",
            wrap=True, ha="center", fontsize=12)

plt.savefig('output.png')
plt.show()


C:\Users\79823\AppData\Local\Temp\ipykernel_20948\2379539049.py:56: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
plt.savefig('support_and_bounded_vectors.png', bbox_inches='tight')

<Figure size 640x480 with 0 Axes>


KeyboardInterrupt

